In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

fidelity_tickers = pd.read_csv("fidelity_ipo_tickers.csv")
fidelity_tickers = fidelity_tickers[fidelity_tickers["Deal Type"] == "IPO"]
fidelity_tickers.head()

,Offering,Pricing Date,Price,Symbol,Deal Type,Bookrunner
0,Anteris Technologies Global Corp,2024-12-12,$6.00,AVR,IPO,"TD Cowen, Barclays, Cantor Fitzgerald TD Cowen"
2,"zSpace, Inc.",2024-12-04,$5.00,ZSPC,IPO,"Roth Capital, Northland Capital Markets Roth C..."
3,Brazil Potash Corp.,2024-11-26,$15.00,GRO,IPO,"Cantor Fitzgerald, Bradesco BBI Cantor Fitzgerald"
6,"Septerna, Inc.",2024-10-24,$18.00,SEPN,IPO,"JP Morgan, TD Cowen, Cantor, Wells Fargo Canto..."
7,Ingram Micro Holding Corp,2024-10-23,$22.00,INGM,IPO,"Morgan Stanley, Goldman Sachs, JP Morgan Everc..."


In [2]:
def get_first_day_close(symbol):
    ticker = yf.Ticker(symbol)
    data = ticker.history(period="10d")

    if data.empty:
        return np.nan

    return data.iloc[0]["Close"]

fidelity_tickers["First Day Close"] = fidelity_tickers["Symbol"].apply(get_first_day_close)
fidelity_tickers.head()

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: CRGX"}}}
$CRGX: possibly delisted; no price data found  (period=10d) (Yahoo error = "No data found, symbol may be delisted")
$RYZB: possibly delisted; no price data found  (period=10d) (Yahoo error = "No data found, symbol may be delisted")
$MYNA: possibly delisted; no price data found  (period=10d) (Yahoo error = "No data found, symbol may be delisted")
$RGF: possibly delisted; no price data found  (period=10d)
$HRT: possibly delisted; no price data found  (period=10d) (Yahoo error = "No data found, symbol may be delisted")
$DTC: possibly delisted; no price data found  (period=10d) (Yahoo error = "No data found, symbol may be delisted")
$INFA: possibly delisted; no price data found  (period=10d)
$ENFN: possibly delisted; no price data found  (period=10d) (Yahoo error = "No data found, symbol may be delisted")
$AVDX: possibly delisted; no price data found  (period=10d) 

,Offering,Pricing Date,Price,Symbol,Deal Type,Bookrunner,First Day Close
0,Anteris Technologies Global Corp,2024-12-12,$6.00,AVR,IPO,"TD Cowen, Barclays, Cantor Fitzgerald TD Cowen",4.730000
2,"zSpace, Inc.",2024-12-04,$5.00,ZSPC,IPO,"Roth Capital, Northland Capital Markets Roth C...",0.498000
3,Brazil Potash Corp.,2024-11-26,$15.00,GRO,IPO,"Cantor Fitzgerald, Bradesco BBI Cantor Fitzgerald",2.040000
6,"Septerna, Inc.",2024-10-24,$18.00,SEPN,IPO,"JP Morgan, TD Cowen, Cantor, Wells Fargo Canto...",27.180000
7,Ingram Micro Holding Corp,2024-10-23,$22.00,INGM,IPO,"Morgan Stanley, Goldman Sachs, JP Morgan Everc...",21.629999


In [3]:
def first_day_return(row):
    symbol = row["Symbol"]
    
    offer_price= float(str(row["Price"]).replace('$', '').replace(',', ''))

    close_price = row["First Day Close"]

    if pd.isna(close_price) or offer_price <= 0:
        return np.nan

    return (close_price - offer_price) / offer_price

fidelity = fidelity_tickers.dropna().copy()
fidelity["First Day Return"] = fidelity.apply(first_day_return, axis=1)
fidelity.head()

,Offering,Pricing Date,Price,Symbol,Deal Type,Bookrunner,First Day Close,First Day Return
0,Anteris Technologies Global Corp,2024-12-12,$6.00,AVR,IPO,"TD Cowen, Barclays, Cantor Fitzgerald TD Cowen",4.730000,-0.211667
2,"zSpace, Inc.",2024-12-04,$5.00,ZSPC,IPO,"Roth Capital, Northland Capital Markets Roth C...",0.498000,-0.900400
3,Brazil Potash Corp.,2024-11-26,$15.00,GRO,IPO,"Cantor Fitzgerald, Bradesco BBI Cantor Fitzgerald",2.040000,-0.864000
6,"Septerna, Inc.",2024-10-24,$18.00,SEPN,IPO,"JP Morgan, TD Cowen, Cantor, Wells Fargo Canto...",27.180000,0.510000
7,Ingram Micro Holding Corp,2024-10-23,$22.00,INGM,IPO,"Morgan Stanley, Goldman Sachs, JP Morgan Everc...",21.629999,-0.016818


In [4]:
ipo_market_summary_data = {
    1980: {"num_ipos": 71, "mean_first_day_equal": 0.143, "mean_first_day_proceeds": 0.200, "median_first_day": 0.069, "money_left_bn": 0.18, "proceeds_bn": 0.91},
    1981: {"num_ipos": 192, "mean_first_day_equal": 0.059, "mean_first_day_proceeds": 0.057, "median_first_day": 0.003, "money_left_bn": 0.13, "proceeds_bn": 2.31},
    1982: {"num_ipos": 77, "mean_first_day_equal": 0.110, "mean_first_day_proceeds": 0.133, "median_first_day": 0.037, "money_left_bn": 0.13, "proceeds_bn": 1.00},
    1983: {"num_ipos": 451, "mean_first_day_equal": 0.099, "mean_first_day_proceeds": 0.094, "median_first_day": 0.025, "money_left_bn": 0.84, "proceeds_bn": 8.89},
    1984: {"num_ipos": 171, "mean_first_day_equal": 0.037, "mean_first_day_proceeds": 0.025, "median_first_day": 0.000, "money_left_bn": 0.05, "proceeds_bn": 2.02},
    1985: {"num_ipos": 186, "mean_first_day_equal": 0.064, "mean_first_day_proceeds": 0.056, "median_first_day": 0.028, "money_left_bn": 0.23, "proceeds_bn": 4.09},
    1986: {"num_ipos": 393, "mean_first_day_equal": 0.061, "mean_first_day_proceeds": 0.051, "median_first_day": 0.011, "money_left_bn": 0.68, "proceeds_bn": 13.40},
    1987: {"num_ipos": 285, "mean_first_day_equal": 0.056, "mean_first_day_proceeds": 0.057, "median_first_day": 0.013, "money_left_bn": 0.66, "proceeds_bn": 11.68},
    1988: {"num_ipos": 105, "mean_first_day_equal": 0.055, "mean_first_day_proceeds": 0.034, "median_first_day": 0.023, "money_left_bn": 0.13, "proceeds_bn": 3.88},
    1989: {"num_ipos": 116, "mean_first_day_equal": 0.080, "mean_first_day_proceeds": 0.047, "median_first_day": 0.040, "money_left_bn": 0.27, "proceeds_bn": 5.81},
    1990: {"num_ipos": 110, "mean_first_day_equal": 0.108, "mean_first_day_proceeds": 0.081, "median_first_day": 0.056, "money_left_bn": 0.34, "proceeds_bn": 4.24},
    1991: {"num_ipos": 286, "mean_first_day_equal": 0.119, "mean_first_day_proceeds": 0.097, "median_first_day": 0.075, "money_left_bn": 1.50, "proceeds_bn": 15.39},
    1992: {"num_ipos": 412, "mean_first_day_equal": 0.103, "mean_first_day_proceeds": 0.080, "median_first_day": 0.044, "money_left_bn": 1.82, "proceeds_bn": 22.69},
    1993: {"num_ipos": 510, "mean_first_day_equal": 0.127, "mean_first_day_proceeds": 0.112, "median_first_day": 0.063, "money_left_bn": 3.52, "proceeds_bn": 31.44},
    1994: {"num_ipos": 402, "mean_first_day_equal": 0.096, "mean_first_day_proceeds": 0.083, "median_first_day": 0.042, "money_left_bn": 1.43, "proceeds_bn": 17.18},
    1995: {"num_ipos": 462, "mean_first_day_equal": 0.214, "mean_first_day_proceeds": 0.175, "median_first_day": 0.132, "money_left_bn": 4.90, "proceeds_bn": 27.93},
    1996: {"num_ipos": 677, "mean_first_day_equal": 0.172, "mean_first_day_proceeds": 0.161, "median_first_day": 0.100, "money_left_bn": 6.76, "proceeds_bn": 42.05},
    1997: {"num_ipos": 474, "mean_first_day_equal": 0.140, "mean_first_day_proceeds": 0.144, "median_first_day": 0.094, "money_left_bn": 4.56, "proceeds_bn": 31.76},
    1998: {"num_ipos": 283, "mean_first_day_equal": 0.219, "mean_first_day_proceeds": 0.156, "median_first_day": 0.089, "money_left_bn": 5.25, "proceeds_bn": 33.66},
    1999: {"num_ipos": 476, "mean_first_day_equal": 0.712, "mean_first_day_proceeds": 0.574, "median_first_day": 0.375, "money_left_bn": 37.11, "proceeds_bn": 64.67},
    2000: {"num_ipos": 380, "mean_first_day_equal": 0.563, "mean_first_day_proceeds": 0.458, "median_first_day": 0.279, "money_left_bn": 29.68, "proceeds_bn": 64.80},
    2001: {"num_ipos": 80, "mean_first_day_equal": 0.140, "mean_first_day_proceeds": 0.084, "median_first_day": 0.102, "money_left_bn": 2.97, "proceeds_bn": 35.29},
    2002: {"num_ipos": 66, "mean_first_day_equal": 0.091, "mean_first_day_proceeds": 0.051, "median_first_day": 0.082, "money_left_bn": 1.13, "proceeds_bn": 22.03},
    2003: {"num_ipos": 63, "mean_first_day_equal": 0.117, "mean_first_day_proceeds": 0.104, "median_first_day": 0.087, "money_left_bn": 1.00, "proceeds_bn": 9.54},
    2004: {"num_ipos": 173, "mean_first_day_equal": 0.123, "mean_first_day_proceeds": 0.124, "median_first_day": 0.071, "money_left_bn": 3.86, "proceeds_bn": 31.19},
    2005: {"num_ipos": 159, "mean_first_day_equal": 0.103, "mean_first_day_proceeds": 0.093, "median_first_day": 0.058, "money_left_bn": 2.64, "proceeds_bn": 28.23},
    2006: {"num_ipos": 157, "mean_first_day_equal": 0.121, "mean_first_day_proceeds": 0.130, "median_first_day": 0.056, "money_left_bn": 3.95, "proceeds_bn": 30.48},
    2007: {"num_ipos": 159, "mean_first_day_equal": 0.140, "mean_first_day_proceeds": 0.139, "median_first_day": 0.068, "money_left_bn": 4.95, "proceeds_bn": 35.66},
    2008: {"num_ipos": 21, "mean_first_day_equal": 0.057, "mean_first_day_proceeds": 0.247, "median_first_day": -0.017, "money_left_bn": 5.63, "proceeds_bn": 22.76},
    2009: {"num_ipos": 41, "mean_first_day_equal": 0.098, "mean_first_day_proceeds": 0.111, "median_first_day": 0.057, "money_left_bn": 1.46, "proceeds_bn": 13.17},
    2010: {"num_ipos": 91, "mean_first_day_equal": 0.094, "mean_first_day_proceeds": 0.062, "median_first_day": 0.031, "money_left_bn": 1.84, "proceeds_bn": 29.82},
    2011: {"num_ipos": 81, "mean_first_day_equal": 0.139, "mean_first_day_proceeds": 0.130, "median_first_day": 0.085, "money_left_bn": 3.51, "proceeds_bn": 26.97},
    2012: {"num_ipos": 93, "mean_first_day_equal": 0.177, "mean_first_day_proceeds": 0.089, "median_first_day": 0.111, "money_left_bn": 2.75, "proceeds_bn": 31.11},
    2013: {"num_ipos": 158, "mean_first_day_equal": 0.209, "mean_first_day_proceeds": 0.190, "median_first_day": 0.130, "money_left_bn": 7.89, "proceeds_bn": 41.56},
    2014: {"num_ipos": 206, "mean_first_day_equal": 0.155, "mean_first_day_proceeds": 0.128, "median_first_day": 0.058, "money_left_bn": 5.40, "proceeds_bn": 42.20},
    2015: {"num_ipos": 118, "mean_first_day_equal": 0.192, "mean_first_day_proceeds": 0.189, "median_first_day": 0.103, "money_left_bn": 4.16, "proceeds_bn": 22.00},
    2016: {"num_ipos": 75, "mean_first_day_equal": 0.145, "mean_first_day_proceeds": 0.142, "median_first_day": 0.050, "money_left_bn": 1.77, "proceeds_bn": 12.52},
    2017: {"num_ipos": 106, "mean_first_day_equal": 0.129, "mean_first_day_proceeds": 0.160, "median_first_day": 0.090, "money_left_bn": 3.68, "proceeds_bn": 22.98},
    2018: {"num_ipos": 134, "mean_first_day_equal": 0.186, "mean_first_day_proceeds": 0.191, "median_first_day": 0.116, "money_left_bn": 6.39, "proceeds_bn": 33.47},
    2019: {"num_ipos": 113, "mean_first_day_equal": 0.235, "mean_first_day_proceeds": 0.176, "median_first_day": 0.179, "money_left_bn": 6.95, "proceeds_bn": 39.28},
    2020: {"num_ipos": 165, "mean_first_day_equal": 0.416, "mean_first_day_proceeds": 0.479, "median_first_day": 0.262, "money_left_bn": 29.66, "proceeds_bn": 61.86},
    2021: {"num_ipos": 311, "mean_first_day_equal": 0.321, "mean_first_day_proceeds": 0.240, "median_first_day": 0.170, "money_left_bn": 28.65, "proceeds_bn": 119.36},
    2022: {"num_ipos": 38, "mean_first_day_equal": 0.489, "mean_first_day_proceeds": 0.142, "median_first_day": 0.093, "money_left_bn": 0.99, "proceeds_bn": 6.99},
    2023: {"num_ipos": 54, "mean_first_day_equal": 0.119, "mean_first_day_proceeds": 0.161, "median_first_day": -0.005, "money_left_bn": 1.92, "proceeds_bn": 11.92},
    2024: {"num_ipos": 72, "mean_first_day_equal": 0.153, "mean_first_day_proceeds": 0.181, "median_first_day": 0.072, "money_left_bn": 3.72, "proceeds_bn": 20.49},
    2025: {"num_ipos": 90, "mean_first_day_equal": 0.293, "mean_first_day_proceeds": 0.336, "median_first_day": 0.137, "money_left_bn": 13.11, "proceeds_bn": 38.97},
}


In [5]:
def get_market_cap(symbol):
    try:
        return yf.Ticker(symbol).info.get("marketCap", np.nan)
    except:
        return np.nan
    
fidelity["market_cap"] = fidelity["Symbol"].apply(get_market_cap)


In [7]:
fidelity.head()

,Offering,Pricing Date,Price,Symbol,Deal Type,Bookrunner,First Day Close,First Day Return,market_cap
0,Anteris Technologies Global Corp,2024-12-12,$6.00,AVR,IPO,"TD Cowen, Barclays, Cantor Fitzgerald TD Cowen",4.730000,-0.211667,216287232
2,"zSpace, Inc.",2024-12-04,$5.00,ZSPC,IPO,"Roth Capital, Northland Capital Markets Roth C...",0.498000,-0.900400,14002133
3,Brazil Potash Corp.,2024-11-26,$15.00,GRO,IPO,"Cantor Fitzgerald, Bradesco BBI Cantor Fitzgerald",2.040000,-0.864000,101931136
6,"Septerna, Inc.",2024-10-24,$18.00,SEPN,IPO,"JP Morgan, TD Cowen, Cantor, Wells Fargo Canto...",27.180000,0.510000,1270691584
7,Ingram Micro Holding Corp,2024-10-23,$22.00,INGM,IPO,"Morgan Stanley, Goldman Sachs, JP Morgan Everc...",21.629999,-0.016818,5181001728


In [9]:
fidelity["Pricing Date"] = pd.to_datetime(fidelity['Pricing Date'])

clean_df = fidelity.dropna(subset=["market_cap", "First Day Return"]).copy()
clean_df['Year'] = clean_df['Pricing Date'].dt.year

# Calculate Equal Weighted Mean
eq_weighted = clean_df.groupby('Year')['First Day Return'].mean().reset_index()
eq_weighted.columns = ['Year', 'Fidelity_Equal_Weight']

# Calculate Proceeds Weighted Mean (using your market_cap proxy)
def weighted_avg(group):
    return np.average(group["First Day Return"], weights=group["market_cap"])

proc_weighted = clean_df.groupby('Year').apply(weighted_avg).reset_index()
proc_weighted.columns = ['Year', 'Fidelity_Proceeds_Weight']

# Merge Fidelity results together
fidelity_summary = eq_weighted.merge(proc_weighted, on='Year')

# Convert dictionary to DataFrame
market_stats = pd.DataFrame.from_dict(ipo_market_summary_data, orient='index')
market_stats = market_stats.reset_index().rename(columns={'index': 'Year'})

# Final Merge
final_table = fidelity_summary.merge(
    market_stats[['Year', 'mean_first_day_equal', 'mean_first_day_proceeds']], 
    on='Year', 
    how='left'
)

# Calculate Alphas (Fidelity Return - Market Return)
final_table['Equal_Weight_Alpha'] = (
    final_table['Fidelity_Equal_Weight'] - final_table['mean_first_day_equal']
)

final_table['Proceeds_Weight_Alpha'] = (
    final_table['Fidelity_Proceeds_Weight'] - final_table['mean_first_day_proceeds']
)

# Rename columns for clarity as requested
final_table = final_table.rename(columns={
    'mean_first_day_equal': 'Market_Equal_Weight',
    'mean_first_day_proceeds': 'Market_Proceeds_Weight'
})

final_table

/var/folders/9y/y72wwkf902j5hylgc5yx8bmm0000gn/T/ipykernel_90409/1049749341.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  proc_weighted = clean_df.groupby('Year').apply(weighted_avg).reset_index()


,Year,Fidelity_Equal_Weight,Fidelity_Proceeds_Weight,Market_Equal_Weight,Market_Proceeds_Weight,Equal_Weight_Alpha,Proceeds_Weight_Alpha
0,2016,2.219362,6.183910,0.145,0.142,2.074362,6.041910
1,2017,0.606059,3.975109,0.129,0.160,0.477059,3.815109
2,2018,0.487651,5.431239,0.186,0.191,0.301651,5.240239
3,2019,0.244766,0.951811,0.235,0.176,0.009766,0.775811
4,2020,0.434694,0.862465,0.416,0.479,0.018694,0.383465
5,2021,-0.091439,4.652690,0.321,0.240,-0.412439,4.412690
6,2022,-0.371526,0.030454,0.489,0.142,-0.860526,-0.111546
7,2023,0.846450,1.451821,0.119,0.161,0.727450,1.290821
8,2024,0.278888,2.296685,0.153,0.181,0.125888,2.115685


In [10]:
print("Average Equal Weight diff(fidelity - market):", final_table["Equal_Weight_Alpha"].mean())
print("Average Proceed Weight diff(fidelity - market):", final_table["Proceeds_Weight_Alpha"].mean())

Average Equal Weight diff(fidelity - market): 0.273544941652254
Average Proceed Weight diff(fidelity - market): 2.66268728089648
